# بسم الله الرحمن الرحیم

# محمد مهدی شفیقی - پروژه نهایی درس مباحث ویژه

# تحلیل شبکه همکاری علمی CA-HepTh

این پروژه به تحلیل ویژگی‌های ساختاری و دینامیکی شبکه هم‌نویسندگی در حوزه فیزیک انرژی بالا می‌پردازد.

In [2]:
import sys
sys.path.append('../src') # اضافه کردن پوشه src به مسیر پایتون
from data_parser import build_temporal_graphs
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import community as community_louvain
import pandas as pd

# ساخت گراف‌ها با فراخوانی تابع
data_path = '../data/cit-HepTh-abstracts'
temporal_graphs, G_final = build_temporal_graphs(data_path)

print(f"گراف نهایی با {G_final.number_of_nodes()} گره و {G_final.number_of_edges()} یال ساخته شد.")

گراف نهایی با 70476 گره و 226489 یال ساخته شد.


## مرحله اول: تحلیل شاخص‌های پایه
در این بخش، شاخص‌های اصلی شبکه را هم در حالت پایا (برای کل دوره) و هم به صورت زمانی تحلیل می‌کنیم.

In [ ]:
# محاسبه شاخص ها برای گراف نهایی (G_final)

# 1. Average Degree
avg_degree = sum(dict(G_final.degree()).values()) / G_final.number_of_nodes()
print(f"Average Degree: {avg_degree:.2f}")

# 2. Graph Density
density = nx.density(G_final)
print(f"Graph Density: {density:.6f}")

# 3. Connected Components
# برای سادگی کار با بزرگترین مولفه همبند
largest_cc_nodes = max(nx.connected_components(G_final), key=len)
G_main = G_final.subgraph(largest_cc_nodes)
print(f"Number of connected components: {nx.number_connected_components(G_final)}")
print(f"Size of largest component: {G_main.number_of_nodes()} nodes")


# 4. Network Diameter & Average Path Length (روی بزرگترین مولفه همبند)
# هشدار: محاسبه قطر خیلی زمان‌بر بود.
diameter = nx.diameter(G_main)
print(f"Network Diameter: {diameter}")
avg_path_length = nx.average_shortest_path_length(G_main)
print(f"Average Path Length: {avg_path_length:.2f}")

# 5. Average Clustering Coefficient
avg_clustering = nx.average_clustering(G_final)
print(f"Average Clustering Coefficient: {avg_clustering:.4f}")

# 6. Centrality Measures (نمونه برای چند گره)
# این محاسبات برای کل گراف بسیار سنگین هستند، معمولا برای گره های مهم محاسبه می شوند
closeness = nx.closeness_centrality(G_main)
betweenness = nx.betweenness_centrality(G_main)
pagerank = nx.pagerank(G_final)
hits = nx.hits(G_final)

print("Top 5 nodes by PageRank:")
sorted_pagerank = sorted(pagerank.items(), key=lambda item: item[1], reverse=True)
for i in range(5):
    print(f"{i+1}. {sorted_pagerank[i][0]}: {sorted_pagerank[i][1]:.6f}")

Average Degree: 6.43
Graph Density: 0.000091
Number of connected components: 4014
Size of largest component: 57633 nodes


به این دلیل که محاسبات بعضی از ویژگی ها مثل 

Closeness , Diameter , PageRank

بسیار طولانی شد در ورژن ، اول برای بررسی سلامت کد آنها را کامنت کردیم.

# تحلیل زمانی (Temporal Analysis)


years = sorted(temporal_graphs.keys())
avg_degrees_time = []
densities_time = []
avg_clustering_time = []

for year in years:
    graph_snapshot = temporal_graphs[year]
    if graph_snapshot.number_of_nodes() > 0:
        avg_degrees_time.append(sum(dict(graph_snapshot.degree()).values()) / graph_snapshot.number_of_nodes())
        densities_time.append(nx.density(graph_snapshot))
        avg_clustering_time.append(nx.average_clustering(graph_snapshot))
    else:
        avg_degrees_time.append(0)
        densities_time.append(0)
        avg_clustering_time.append(0)

# رسم نمودار تغییرات
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(years, avg_degrees_time, 'b-o')
plt.title("Average Degree Over Time")
plt.xlabel("Year")
plt.ylabel("Average Degree")

plt.subplot(2, 2, 2)
plt.plot(years, densities_time, 'r-o')
plt.title("Density Over Time")
plt.xlabel("Year")
plt.ylabel("Density")

plt.subplot(2, 2, 3)
plt.plot(years, avg_clustering_time, 'g-o')
plt.title("Average Clustering Coefficient Over Time")
plt.xlabel("Year")
plt.ylabel("Clustering")

plt.tight_layout()
plt.show()